In [1]:
#檢查有無安裝好spark (我使用pyspark)
sc

<SparkContext master=local[2] appName=PySparkShell>

//一開始直接讀textfile資料會讀失敗
需要先設定好header、delimiter、inferSchema、csv路徑

//以下為讀進textfile轉為dataframe
經過以下修改後才可以把textfile讀成正確的dataframe

In [9]:
#先讀資料進來
df = spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("inferSchema", "true") \
    .csv("C://Users//user//Desktop//household_power_consumption//household_power_consumption.txt")

df.show()

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|              4.216|                0.418|234.840|          18.400|         0.000|         1.000|          17.0|
|16/12/2006|17:25:00|              5.360|                0.436|233.630|          23.000|         0.000|         1.000|          16.0|
|16/12/2006|17:26:00|              5.374|                0.498|233.290|          23.000|         0.000|         2.000|          17.0|
|16/12/2006|17:27:00|              5.388|                0.502|233.740|          23.000|         0.000|         1.000|          17.0|
|16/12/2006|17:28:00|              3.666|                0.528

In [21]:
#把該import的都先import進來
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession


【30pt】作業Part1: 算出以下四個header(column)個別的min、max、count值 

In [22]:
#Global_active_power
df_new = df.filter(df.Global_active_power != '?') #把這個column中的'?'值給過濾掉才不會輸出'?'
df_new.agg(min(col("Global_active_power")), max(col("Global_active_power")), count(col("Global_active_power"))).show()

#Global_reactive_power
df_new2 = df.filter(df.Voltage != '?')
df_new2.agg(min(col("Global_reactive_power")), max(col("Global_reactive_power")), count(col("Global_reactive_power"))).show()

#Voltage
df_new3 = df.filter(df.Voltage != '?')
df_new3.agg(min(col("Voltage")), max(col("Voltage")), count(col("Voltage"))).show()

#Global_intensity
df_new4 = df.filter(df.Global_intensity != '?')
df_new4.agg(min(col("Global_intensity")), max(col("Global_intensity")), count(col("Global_intensity"))).show()

+------------------------+------------------------+--------------------------+
|min(Global_active_power)|max(Global_active_power)|count(Global_active_power)|
+------------------------+------------------------+--------------------------+
|                   0.076|                   9.994|                   2049280|
+------------------------+------------------------+--------------------------+

+--------------------------+--------------------------+----------------------------+
|min(Global_reactive_power)|max(Global_reactive_power)|count(Global_reactive_power)|
+--------------------------+--------------------------+----------------------------+
|                     0.000|                     1.390|                     2049280|
+--------------------------+--------------------------+----------------------------+

+------------+------------+--------------+
|min(Voltage)|max(Voltage)|count(Voltage)|
+------------+------------+--------------+
|     223.200|     254.150|       2049280|
+-----

【30pt】作業Part2: 輸出上述這些header(column)的mean、standard deviation

In [23]:
#Global_active_power
df_new5 = df.filter(df.Global_active_power != '?')
df_new5.select(mean("Global_active_power"), stddev("Global_active_power")).show(truncate=False)

#Global_reactive_power
df_new6 = df.filter(df.Global_reactive_power != '?')
df_new6.select(mean("Global_reactive_power"), stddev("Global_reactive_power")).show(truncate=False)

#Voltage
df_new7 = df.filter(df.Voltage != '?')
df_new7.select(mean("Voltage"), stddev("Voltage")).show(truncate=False)

#Global_intensity
df_new8 = df.filter(df.Global_intensity != '?')
df_new8.select(mean("Global_intensity"), stddev("Global_intensity")).show(truncate=False)

+------------------------+--------------------------------+
|avg(Global_active_power)|stddev_samp(Global_active_power)|
+------------------------+--------------------------------+
|1.0916150365007122      |1.0572941610939701              |
+------------------------+--------------------------------+

+--------------------------+----------------------------------+
|avg(Global_reactive_power)|stddev_samp(Global_reactive_power)|
+--------------------------+----------------------------------+
|0.12371447630388838       |0.1127219795507155                |
+--------------------------+----------------------------------+

+-----------------+--------------------+
|avg(Voltage)     |stddev_samp(Voltage)|
+-----------------+--------------------+
|240.8398579745544|3.2399866790098937  |
+-----------------+--------------------+

+---------------------+-----------------------------+
|avg(Global_intensity)|stddev_samp(Global_intensity)|
+---------------------+-----------------------------+
|4.6277593

【40pt】作業Part3: 執行min-max normalization來求出上述header(column)的normalized output

//開始處裡Min-Max Normalization

In [24]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline


#1.original dataset

df = spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("inferSchema", "true") \
    .csv("C://Users//user//Desktop//household_power_consumption//household_power_consumption.txt")

#抓出指定的column
df_choose = df.select("Global_active_power", "Global_reactive_power", "Voltage", "Global_intensity")

#因為VectorAssembler不接受string型態的input，所以要把每個column的數據型態從String轉型為數字
df=df_choose.withColumn("Global_active_power", df.Global_active_power.astype("float"))
df=df.withColumn("Global_reactive_power", df.Global_reactive_power.astype("float"))
df=df.withColumn("Voltage", df.Voltage.astype("float"))
df=df.withColumn("Global_intensity", df.Global_intensity.astype("float"))

#df.printSchema()
#df.show(5)

#把每個column分開來獨立成一個dataframe
df1 = df.select("Global_active_power")
df2 = df.select("Global_reactive_power")
df3 = df.select("Voltage")
df4 = df.select("Global_intensity")


#2.Vector assembled set of features 

assembler1 = VectorAssembler(inputCols=["Global_active_power"], outputCol="features1", handleInvalid='skip')
output1 = assembler1.transform(df1)

assembler2 = VectorAssembler(inputCols=["Global_reactive_power"], outputCol="features2", handleInvalid='skip')
output2 = assembler2.transform(df2)

assembler3 = VectorAssembler(inputCols=["Voltage"], outputCol="features3", handleInvalid='skip')
output3 = assembler3.transform(df3)

assembler4 = VectorAssembler(inputCols=["Global_intensity"], outputCol="features4", handleInvalid='skip')
output4 = assembler4.transform(df4)


#3. Applying MinMaxScaler to my assembled features 

#會有4個output
scaler = MinMaxScaler(inputCol="features1", outputCol="normalized global active power")
# rescale each feature to range [min, max].
scaledData1 = scaler.fit(output1).transform(output1)
scaledData1.select("normalized global active power").show(5)

scaler = MinMaxScaler(inputCol="features2", outputCol="normalized global reactive power")
scaledData2 = scaler.fit(output2).transform(output2)
scaledData2.select("normalized global reactive power").show(5)

scaler = MinMaxScaler(inputCol="features3", outputCol="normalized voltage")
scaledData3 = scaler.fit(output3).transform(output3)
scaledData3.select("normalized voltage").show(5)

scaler = MinMaxScaler(inputCol="features4", outputCol="normalized global intensity")
scaledData4 = scaler.fit(output4).transform(output4)
scaledData4.select("normalized global intensity").show(5)

+------------------------------+
|normalized global active power|
+------------------------------+
|          [0.3747963225473887]|
|          [0.4783632318374359]|
|          [0.47963065346431...|
|          [0.4808980750911956]|
|          [0.32500452429198...|
+------------------------------+
only showing top 5 rows

+--------------------------------+
|normalized global reactive power|
+--------------------------------+
|            [0.3007194366460509]|
|            [0.3136690604292478]|
|            [0.35827338206620...|
|            [0.3611510643288495]|
|            [0.3798561169588938]|
+--------------------------------+
only showing top 5 rows

+--------------------+
|  normalized voltage|
+--------------------+
|[0.37609048586059...|
|[0.33699544306942...|
|[0.32600960687537...|
| [0.340549582688101]|
|[0.4032309194863383]|
+--------------------+
only showing top 5 rows

+---------------------------+
|normalized global intensity|
+---------------------------+
|       [0.37759